<a href="https://colab.research.google.com/github/ArthurCTLin/Workbook/blob/main/Lecture304_practice01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Connect to TPU
* Edit -> NoteBook settings
* Select TPU from the Hardware Accelerator drop-down
* Run the following script to check the connect situation

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.4.1
Running on TPU  ['10.36.133.26:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.36.133.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.36.133.26:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


### Connect to google drive for loading data

In [22]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
!ls "/content/drive/MyDrive/ColabNotebooks/"

font_data.npy  l304practice.npz  Lecture304_practice01.ipynb


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds

data = np.load('/content/drive/MyDrive/ColabNotebooks/l304practice.npz')
x_train = data['x_train']
y_train = data['y_train']
#y_train = np.array([np.eye(2)[n-1] for n in data['y_train']])
x_test = data['x_test']
y_test = data['y_test']
#y_test = np.array([np.eye(2)[n-1] for n in data['y_test']])

y_train = np.array([n-1 for n in y_train])
y_test = np.array([n-1 for n in y_test])

y_train = np.array([[[1,0],[0,1]][n] for n in y_train])
y_test = np.array([[[1,0],[0,1]][n] for n in y_test])

print('x shape:', x_train.shape)
print('y shape:', y_train.shape)

print('1st sample in x:', x_train[0])
print('1st sample in y:', y_train[0])

from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adadelta

#img_shape = (48,48)

model = Sequential()
model.add(Reshape((48,48,1), input_shape=(48,48)))
model.add(Conv2D(16, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=1,
          validation_data=(x_test, y_test))

print('Performance (training)')
print('Loss: %.5f, Acc: %.5f' % tuple(model.evaluate(x_train, y_train)))
print('Performance (testing)')
print('Loss: %.5f, Acc: %.5f' % tuple(model.evaluate(x_test, y_test)))

p_test = model.predict(x_test)
failedsample = [[img,y,p] for img,y,p in zip(data['x_test'],y_test,p_test) if y.argmax()!=p.argmax()]
print('# of failed test samples:',len(failedsample))

fig = plt.figure(figsize=(10,10), dpi=80)
for i in range(len(failedsample[:100])):
    plt.subplot(10,10,i+1)
    plt.axis('off')
    plt.imshow(failedsample[i][0], cmap='Greys')
    plt.text(0.,0.,'$%d\\to%d$' % (failedsample[i][1].argmax(),failedsample[i][2].argmax()),color='Red',fontsize=15)
plt.show()


x shape: (6400, 48, 48)
y shape: (6400, 2)
1st sample in x: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
1st sample in y: [1 0]
Epoch 1/20
6400/6400 [==============================] - 85s 13ms/step - loss: 21.2192 - accuracy: 0.5703 - val_loss: 9.3221 - val_accuracy: 0.6465
Epoch 2/20
6400/6400 [==============================] - 86s 13ms/step - loss: 12.0207 - accuracy: 0.6331 - val_loss: 6.9748 - val_accuracy: 0.7155
Epoch 3/20
6400/6400 [==============================] - 85s 13ms/step - loss: 9.6438 - accuracy: 0.6883 - val_loss: 5.4990 - val_accuracy: 0.7615
Epoch 4/20
6400/6400 [==============================] - 86s 13ms/step - loss: 7.9070 - accuracy: 0.7283 - val_loss: 4.4655 - val_accuracy: 0.7963
Epoch 5/20
6400/6400 [==============================] - 85s 13ms/step - loss: 6.9019 - accuracy: 0.7544 - val_loss: 3.7378 - val_accuracy: 0.8262
Epoch 6/20
6400/6400 [==============================] - 86s 13ms/